In [0]:
CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.flat') (
  path STRING,
  vendor_name STRING,
  file_name STRING,
  other_vendor_files ARRAY<STRING>,
  text STRING,
  preamble STRING,
  truncated STRING,
  CONSTRAINT flat_pk PRIMARY KEY (path)
)
USING DELTA;

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.flat') AS target
USING (
  WITH flattened AS (
    SELECT
      *,
      concat_ws(
          '\n\n',
          transform(
            try_cast(parsed:document:elements AS ARRAY<VARIANT>),
            element -> try_cast(element:content AS STRING)
          )
        ) AS text,
      concat_ws(
          ' ',
          slice(
            split(
              concat_ws(
                '\n\n',
                transform(
                  try_cast(parsed:document:elements AS ARRAY<VARIANT>),
                  element -> try_cast(element:content AS STRING)
                )
              ),
              ' '
            ),
            1,
            500
          )
        ) AS preamble,
      concat_ws(
          ' ',
          slice(
            split(
              concat_ws(
                '\n\n',
                transform(
                  try_cast(parsed:document:elements AS ARRAY<VARIANT>),
                  element -> try_cast(element:content AS STRING)
                )
              ),
              ' '
            ),
            1,
            10000
          )
        ) AS truncated
    FROM IDENTIFIER(:catalog || '.' || :schema || '.parsed')
  )
  SELECT * FROM flattened
  WHERE text != ''
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN
  INSERT *
;